# Experiment tracking

In this notebook, we use our self-hosted experiment tracker to record some metrics.

In [1]:
import asyncio
import modal
from aim import Run
from track.aim import get_repo
from infra.requirements import freeze, project_packages
from mini.experiment import Experiment

from utils.logging import SimpleLoggingConfig

logging_config = SimpleLoggingConfig().info('track')
logging_config.apply()

# app = modal.App(
#     'aim test',
#     image=image,
# )
run = Experiment('aim test')
run.image = modal.Image.debian_slim().pip_install(*freeze(all=True)).add_local_python_source(*project_packages())


@run.thither
async def emit_some_metrics():
    logging_config.apply()

    repo = get_repo()
    run = Run(repo=repo)
    for x in range(20):
        print(x)
        run.track(x, 'foo')
        run.track(x**2, 'bar')
        await asyncio.sleep(1)
    run.close()

In [2]:
from unittest import mock
from IPython.display import clear_output

with mock.patch('rich.console._is_jupyter', return_value=False), modal.enable_output():
    # Build images, if needed.
    # By entering the experiment as a context, the thither functions will have their images built.
    # We also need to actually call `get_repo`, because it comes from another app, so it won't be built otherwise.
    # This would all happen automatically in the next cell when the function is actually called, but due to Modal's output handling, it's hard to tell whether the function has started and hung or if it's still building.
    async with run():
        get_repo()
clear_output()

In [3]:
async with run():
    await emit_some_metrics()

I 1.6 tr.pa.cl:Patching Aim Client to use Basic auth
I 1.6 tr.pa.cl:Setting bearer token
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
